In [227]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime, timedelta
import time
import csv
from tqdm import tqdm
tqdm.pandas()

## 1. 네이버 뉴스 10개 기사 수집, 전처리 후 저장

In [228]:
u = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}
try:
    rs = requests.get(u, headers=headers)
except:
    print('RETRY-TIME ERROR!')
    time.sleep(3)
    rs = requests.get(u, headers=headers)
    
if rs.status_code != 200:
    print('CONTENT REQUEST FAILED! : ', u)
    "REQUEST_ERROR"
rs.encoding = 'EUC-KR' # html의 인코딩 방식 확인하기
soup = BeautifulSoup(rs.text, 'html.parser')

In [229]:
urls = []
for i in range(1,11):
    url = soup.select_one(f'#main_content > div > div._persist > div.section_headline > ul > li:nth-child({i}) > div.sh_text > a').get_attribute_list('href')[0]
    urls.append(url)

f = open('20231002.csv','w', newline='')
wtr = csv.writer(f)
wtr.writerow(['언론사', '기사제목', '기사본문'])

for url in urls:    
    rs = requests.get(url, headers=headers)
    rs.encoding = 'UTF-8' # html의 인코딩 방식 확인하기
    soup = BeautifulSoup(rs.text, 'html.parser')
    company = soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_top > a > img').get_attribute_list('title')[0]
    title = soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_title').text
    content = soup.select_one('#newsct_article')
    if content != None:
        for i in content.find_all('span'):
            i.decompose()
        content = content.text
    title = re.sub('[^A-Za-z0-9가-힣]', ' ', title)
    content = re.sub('[^A-Za-z0-9가-힣]', ' ', content)
    wtr.writerow([company, title, content])
    
f.close()

## 2. 기사 댓글 수 수집

In [230]:
df = pd.read_csv('20231002.csv', encoding='cp949')

In [231]:
from selenium import webdriver
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
# options.add_argument('headless') #웹페이지 띄우지 않기
drive = webdriver.Chrome(options=options)

In [232]:
comment_counts = []
for url in urls:
    drive.implicitly_wait(10)
    drive.get(url)
    comment_count = drive.find_element(By.CLASS_NAME, 'u_cbox_count').text
    comment_counts.append(comment_count)

df['댓글 수'] = comment_counts

In [233]:
df

,언론사,기사제목,기사본문,댓글 수
0,MBC,엔 달러 환율 150엔 돌파 직후 급락 시장개입 가능성,엔 달러 환율이 현지시간 3일 미국 뉴욕 외환시장에서 장중 한때 달러당 150 ...,1
1,한국경제TV,또 올랐다 경유값 1 700원 돌파,국내 주유소 경유 판매 가격이 약 9개월 만에 L 리터 당 1천700원을 돌파했...,0
2,파이낸셜뉴스,부채 늪 에 빠진 자영업자 대출연체 7조원 넘었다 사상 최대,올 2분기 자영업 대출잔액 1043조 돌파 같은기간 원리금 연체 1조 늘어 빨...,27
3,한국경제,현대차 야심작 아이오닉 7 내년 7월 출격 어떻게 나올까 배성수의 다다IT선,배성수의 다다IT선 137회현대차 아산공장서 내년 7월 아이오닉7 양산미래 자...,69
4,경향신문,정부 전월세 계약 신고할 때 공인중개사 인적 정보 기재 의무화 추진,전월세 계약을 신고할 때 계약을 중개한 공인중개사의 인적 정보를 의무적으로...,4
5,노컷뉴스,단독 전세사기 심각한데 급증하는 보증보험 지급 거절,HUG 5년간 지급 거절 182건 거절 금액만 359억 8300만 원올해 사기...,14
6,뉴스1,8월 생산 30개월 만에 최대폭 경기 반등 조짐 확대 종합2보,산업활동동향 생산 2 2 투자 3 6 소매판매는 0 3 광공업 생산 ...,3
7,경향신문,속보 원 달러 11개월만에 1360원 돌파 달러강세에 급등,원 달러 환율이 연휴를 마치고 장 초반 10원 이상 급등해 1360원선을 돌파했...,174
8,아시아경제,SK에코플랜트 인천공항공사와 업무협약 인천공항 폐기물 재활용한다,SK에코플랜트는 인천국제공항공사와 함께 인천공항 자원순환경제 구축을 위한...,0
9,뉴스1,해수부 제30회 부산 자갈치 축제 서 수산물 안전 홍보 시식행사 진행,숙성 광어 회 도시락 시식행사 수산물 요리법 책자 안전 홍보물 배포 세종 ...,0


In [234]:
df.to_csv('20231002.csv', index=False)